In [1]:
import torch
import torch.nn as nn


In [2]:
from functools import reduce
from operator import __add__
class Conv2dSamePadding(nn.Conv2d):
    def __init__(self,*args,**kwargs):
        super(Conv2dSamePadding, self).__init__(*args, **kwargs)
        self.zero_pad_2d = nn.ZeroPad2d(reduce(__add__,
            [(k // 2 + (k - 2 * (k // 2)) - 1, k // 2) for k in self.kernel_size[::-1]]))

    def forward(self, input):
        return  self._conv_forward(self.zero_pad_2d(input), self.weight, self.bias)

In [3]:
def calc_same_padding(kernel_size, stride, input_size):
    if isinstance(kernel_size, Sequence):
        kernel_size = kernel_size[0]

    if isinstance(stride, Sequence):
        stride = stride[0]

    if isinstance(input_size, Sequence):
        input_size = input_size[0]

    pad = ((stride - 1) * input_size - stride + kernel_size) / 2
    return int(pad)

def replace_conv2d_with_same_padding(m: nn.Module, input_size=512):
    if isinstance(m, nn.Conv2d):
        if m.padding == "same":
            m.padding = calc_same_padding(
                kernel_size=m.kernel_size,
                stride=m.stride,
                input_size=input_size
            )

In [4]:
class FOMO(torch.nn.Module):
    def __init__(self):
        super(FOMO, self).__init__()

        #Reduction
        #3x3 conv stride 2  with 4 out channel 
        self.conv1 = torch.nn.Conv2d(in_channels=3, out_channels=4, kernel_size=3, stride=2, padding=(1,1))
        #3x3 conv stride 2 with 8 out channel 
        self.conv2 = torch.nn.Conv2d(in_channels=4, out_channels=8, kernel_size=3, stride=2, padding=(1,1))
        #3x3 conv stride 2 with 16 out channel 
        self.conv3 = torch.nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, stride=2, padding=(1,1))
        #3x3 conv stride 2 with 32 out channel 
        self.conv4 = torch.nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=2, padding=(1,1))
        #3x3 conv stride 1 with 16 out channel 
        self.conv5 = Conv2dSamePadding(in_channels=32, out_channels=16, kernel_size=3, stride=1)

        self.upsample = torch.nn.Upsample(scale_factor=2, mode='bilinear')

        #Increasing
        self.conv6 = Conv2dSamePadding(in_channels=32, out_channels=8, kernel_size=3, stride=1)
        self.conv7 = Conv2dSamePadding(in_channels=16, out_channels=4, kernel_size=3, stride=1)
        self.conv8 = Conv2dSamePadding(in_channels=8, out_channels=1, kernel_size=1, stride=1)

        

        

    def forward(self, x):
            #Downsample
        out1 = self.conv1(x)
        out2 = self.conv2(out1)
        out3 = self.conv3(out2)

        output = self.conv4(out3)
        
        output = self.upsample(output)
        output = self.conv5(output)
        output = torch.concat(( output, out3), dim=1)
        output = self.upsample(output)
        output = self.conv6(output)
        output = torch.concat(( output, out2), dim=1)
        output = self.upsample(output)
        output = self.conv7(output)
        output = torch.concat(( output, out1), dim=1)
        output = self.conv8(output)
        output = torch.sigmoid(output)

        return output


In [5]:
model = FOMO()
print (model)

FOMO(
  (conv1): Conv2d(3, 4, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (conv2): Conv2d(4, 8, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (conv3): Conv2d(8, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (conv4): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (conv5): Conv2dSamePadding(
    32, 16, kernel_size=(3, 3), stride=(1, 1)
    (zero_pad_2d): ZeroPad2d((1, 1, 1, 1))
  )
  (upsample): Upsample(scale_factor=2.0, mode=bilinear)
  (conv6): Conv2dSamePadding(
    32, 8, kernel_size=(3, 3), stride=(1, 1)
    (zero_pad_2d): ZeroPad2d((1, 1, 1, 1))
  )
  (conv7): Conv2dSamePadding(
    16, 4, kernel_size=(3, 3), stride=(1, 1)
    (zero_pad_2d): ZeroPad2d((1, 1, 1, 1))
  )
  (conv8): Conv2dSamePadding(
    8, 1, kernel_size=(1, 1), stride=(1, 1)
    (zero_pad_2d): ZeroPad2d((0, 0, 0, 0))
  )
)


In [6]:
x = torch.randn(1, 3, 512, 384)
y = model(x)
print (y.shape)

print(y)

torch.save(model.state_dict(), "../../results/FOMO_model.pth")

torch.Size([1, 1, 256, 192])
tensor([[[[0.5738, 0.5363, 0.5423,  ..., 0.5559, 0.5387, 0.5512],
          [0.5400, 0.5425, 0.5847,  ..., 0.5292, 0.5700, 0.5993],
          [0.5336, 0.5987, 0.5347,  ..., 0.4690, 0.4533, 0.5491],
          ...,
          [0.5448, 0.4245, 0.5281,  ..., 0.5098, 0.6201, 0.5260],
          [0.5841, 0.5855, 0.4676,  ..., 0.5059, 0.5337, 0.5219],
          [0.5241, 0.5461, 0.5455,  ..., 0.4538, 0.4654, 0.6418]]]],
       grad_fn=<SigmoidBackward0>)


In [7]:
import torch
import blobconverter
from onnxsim import simplify
import onnx


net = FOMO()
net.load_state_dict(torch.load("../../results/FOMO_model.pth"))


onnx_file = "../../results/FOMO_model.onnx"
torch.onnx.export(
    net,
    x,
    onnx_file,
    opset_version=12,
    do_constant_folding=True,
)


onnx_simplified_path = "../../results/FOMO_model_simple.onnx"
onnx_model =  onnx.load(onnx_file)
model_simp, check = simplify(onnx_model)
onnx.save(model_simp, onnx_simplified_path)



blobconverter.from_onnx(
    model=onnx_file,
    data_type="FP16",
    shaves=8,
    use_cache=False,
    output_dir="../../results",
    optimizer_params=[]
)

/home/mnadar2s/anaconda3/envs/rnd/lib/python3.10/site-packages/torch/onnx/_internal/jit_utils.py:258: UserWarning: Constant folding - Only steps=1 can be constant folded for opset >= 10 onnx::Slice op. Constant folding not applied. (Triggered internally at ../torch/csrc/jit/passes/onnx/constant_fold.cpp:179.)
  _C._jit_pass_onnx_node_shape_type_inference(node, params_dict, opset_version)
/home/mnadar2s/anaconda3/envs/rnd/lib/python3.10/site-packages/torch/onnx/utils.py:687: UserWarning: Constant folding - Only steps=1 can be constant folded for opset >= 10 onnx::Slice op. Constant folding not applied. (Triggered internally at ../torch/csrc/jit/passes/onnx/constant_fold.cpp:179.)
  _C._jit_pass_onnx_graph_shape_type_inference(
/home/mnadar2s/anaconda3/envs/rnd/lib/python3.10/site-packages/torch/onnx/utils.py:1178: UserWarning: Constant folding - Only steps=1 can be constant folded for opset >= 10 onnx::Slice op. Constant folding not applied. (Triggered internally at ../torch/csrc/jit/pa

[==================================================]
Done


PosixPath('../../results/FOMO_model_openvino_2021.4_8shave.blob')